In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import json
from aml_storage import Labels, Block, Descriptor
from utils.builder import DescriptorBuilder
import ase.io
from itertools import product
from utils.clebsh_gordan import ClebschGordanReal
from utils.hamiltonians import fix_pyscf_l1, dense_to_blocks, blocks_to_dense, couple_blocks, decouple_blocks
import matplotlib.pyplot as plt
from utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from utils.hamiltonians import *
import copy

In [2]:
frames = ase.io.read("data/water-hamiltonian/water_coords_1000.xyz",":1000")
for f in frames:
    f.cell = [100,100,100]
    f.positions += 50

In [3]:
jorbs = json.load(open('data/water-hamiltonian/orbs_def2_water.json', "r"))
orbs = {}
zdic = {"O" : 8, "H":1}
for k in jorbs:
    orbs[zdic[k]] = jorbs[k]

In [4]:
hams = np.load("data/water-hamiltonian/water_fock.npy", allow_pickle=True)
for i, f in enumerate(frames):
    hams[i] = fix_pyscf_l1(hams[i], f, orbs)

In [5]:
cg = ClebschGordanReal(4)

In [6]:
rascal_hypers = {
    "interaction_cutoff": 3.5,
    "cutoff_smooth_width": 0.5,
    "max_radial": 3,
    "max_angular": 2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
}

In [7]:
spex = RascalSphericalExpansion(rascal_hypers)
rhoi = spex.compute(frames)

In [8]:
def get_lm_slice(hypers):
    lm_slices = []
    start = 0
    for l in range(hypers["max_angular"] + 1):
        stop = start + 2 * l + 1
        lm_slices.append(slice(start, stop))
        start = stop
    return lm_slices

In [9]:
from rascal.representations import SphericalExpansion

In [10]:
pairs = RascalPairExpansion(rascal_hypers)
gij = pairs.compute(frames)

In [11]:
def compute_rho0ij(hypers, frames):
    lmax= hypers["max_angular"]
    pairs = RascalPairExpansion(hypers)
    gij = pairs.compute(frames)
    new_sparse = Labels(
    names=["sigma", "L", "nu"],
    values=np.array(
        [
            [1, l, 0]
            for l in range(hypers["max_angular"] + 1)                    
        ],
        dtype=np.int32,
            ),
        )
    blocks=[]
    for idx, block in gij:
        blocks.append(block.copy())
    return Descriptor(new_sparse, blocks)

In [12]:
hypers=rascal_hypers

In [13]:
old_rho0ij=compute_rho0ij(hypers, frames)

In [14]:
# works for just water

In [15]:
def compute_rho0ij_builder(hypers, frames):
    if hypers["compute_gradients"]:
        raise Exception("Pair expansion with gradient is not implemented")

    max_atoms = max([len(f) for f in frames])
    actual_global_species = list(
    map(int, np.unique(np.concatenate([f.numbers for f in frames]))))
    calculator = SphericalExpansion(**hypers)
    manager = calculator.transform(frames)
    info = manager.get_representation_info()
    global_species = list(range(max_atoms))
    
    hypers_ij= copy.deepcopy(hypers)
    hypers_ij["global_species"] = global_species
    hypers_ij["expansion_by_species_method"] = "user defined"
    lm_slices = get_lm_slice(hypers)

    ijframes = []
    for f in frames:
        ijf = f.copy()
        ijf.numbers = global_species[:len(f)]
        ijframes.append(ijf)

    calculator = SphericalExpansion(**hypers_ij)
    gij_expansion=calculator.transform(ijframes).get_features(calculator).reshape(len(ijframes), len(ijf), max_atoms, hypers_ij["max_radial"], -1) #TODO: change for differet len
#     print(gij_expansion.shape)
    
    feat_builder= DescriptorBuilder(["block_type", "L", "nu", "sigma",  "species_i", "species_j"], ["structure", "center_i", "center_j"], ["mu"], ["n"])

    pair_loc=[]
    lmax = hypers["max_angular"]
    for sp_i in actual_global_species:
        for sp_j in actual_global_species:
            center_species_mask = np.where(info[:, 2] == sp_i)[0]
            neighbor_species_mask = np.where(info[:, 2] == sp_j)[0]
            for i, (struct_i, atom_i) in enumerate(info[center_species_mask[:]][...,:-1]):
                for j, (struct_j, atom_j) in enumerate(info[neighbor_species_mask[:]][...,:-1]):
                    if not (struct_i==struct_j):
                        continue
                    if atom_i==atom_j:
                            block_type = 0  # diagonal
                    elif sp_i==sp_j:
                        block_type = 1  # same-species
                    elif sp_j > sp_i:
                        
                        block_type = 2  # different species
                    else:
                        continue
                        
                    if [struct_i, atom_j, atom_i] not in pair_loc:
                        pair_loc.append([struct_i, atom_i, atom_j])

                    for l in range(lmax+1):
                        block_idx=(block_type, l, 0, 1, sp_i, sp_j)
                        if block_idx not in feat_builder.blocks:
                            block = feat_builder.add_block(sparse=block_idx, features=np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T, 
                                components=np.asarray([list(range(-l, l+1))], dtype=np.int32 ).T )  
                            
                        
                            if block_type == 1:
                                block_asym = feat_builder.add_block(sparse=(-1,)+block_idx[1:], features=np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T,
                                components=np.asarray([list(range(-l, l+1))], dtype=np.int32 ).T )                                                    
                        else:                        
                            block = feat_builder.blocks[block_idx]
                            if block_type == 1:
                                block_asym = feat_builder.blocks[(-1,)+block_idx[1:]]

                        block_data =gij_expansion[struct_i][atom_i%3, atom_j%3, :, lm_slices[l]].T #TODO: change for not water
                        #POSSIBLE replacement:(atom_i-sum(info[:struct_i,:].axis=1))%len(info[np.where(info[:,:,0]==struct_i)])
                        if block_type == 1:
                            if atom_i%3 <=atom_j%3:
                                block_data_ji = gij_expansion[struct_i][atom_j%3, atom_i%3, :, lm_slices[l]].T                  
                                block.add_samples(labels=np.asarray([[struct_i, atom_i%3, atom_j%3]], dtype=np.int32), data=(block_data+block_data_ji).reshape((1,-1,block_data.shape[1]))/np.sqrt(2) )
                                block_asym.add_samples(labels=np.asarray([[struct_i, atom_i%3, atom_j%3]], dtype=np.int32), data=(block_data-block_data_ji).reshape((1,-1,block_data.shape[1]))/np.sqrt(2) )
                        else:
#                       
                            block.add_samples(labels=np.asarray([[struct_i, atom_i%3, atom_j%3]], dtype=np.int32), data=block_data.reshape(1, -1, block_data.shape[1]))
    #                     

    return feat_builder.build()

In [16]:
rho0ij=compute_rho0ij_builder(hypers, frames)

In [17]:
block_type=2

In [18]:
rho0ij.sparse

Labels([( 0, 0, 0, 1, 1, 1), ( 0, 1, 0, 1, 1, 1), ( 0, 2, 0, 1, 1, 1),
        ( 1, 0, 0, 1, 1, 1), (-1, 0, 0, 1, 1, 1), ( 1, 1, 0, 1, 1, 1),
        (-1, 1, 0, 1, 1, 1), ( 1, 2, 0, 1, 1, 1), (-1, 2, 0, 1, 1, 1),
        ( 2, 0, 0, 1, 1, 8), ( 2, 1, 0, 1, 1, 8), ( 2, 2, 0, 1, 1, 8),
        ( 0, 0, 0, 1, 8, 8), ( 0, 1, 0, 1, 8, 8), ( 0, 2, 0, 1, 8, 8)],
       dtype=[('block_type', '<i4'), ('L', '<i4'), ('nu', '<i4'), ('sigma', '<i4'), ('species_i', '<i4'), ('species_j', '<i4')])

In [19]:
block_type=2; L=0; nu=0; sigma=1; species_i=1; species_j=8
rho0ij.block(block_type=2, L=1, nu=0, sigma=1, species_i=1, species_j=8).values

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-8.58231353e-03,  1.96977105e-04,  5.09511459e-04]],

       [[-7.38695871e-03, -1.02626050e-03,  5.52765124e-04],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-7.22540880e-04, -1.00381658e-04,  5.40676366e-05]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-8.57290216e-03,  1.85805842e-04,  5.10210964e-04]],

       ...,

       [[-7.01020074e-03, -8.16594735e-04,  5.12134460e-04],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 2.87243510e-03,  3.34600315e-04, -2.09847485e-04]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-6.32615441e-03, -1.99777569e-03,  5.36805246e-04]],

       [[-8.05392223e-03, -2.24990376e-04,  5.22726542e-04],
 

In [20]:
rho0ij.block(block_type=2, L=1, nu=0, sigma=1, species_i=8, species_j=1).values

#this shouldnt work because we ordered the species in increasing atomic number 

AmlError: invalid parameter: 0 blocks matched the selection (block_type = 2, L = 1, nu = 0, sigma = 1, species_i = 8, species_j = 1), expected only one

## Some tests you can do-
- for even L's block_type -1 should be zero 
- for odd L's block_type 1 should be zero 
- block_type=0 - all values should be the same
- block_type=2, interchanging center and neighbor species should give equivalent results

In [ ]:
rho0ij.block(block_type=2, L=0, nu=0, sigma=1, species_i=8, species_j=1).values

In [ ]:
# to verify with the rascal spherical expansion- find indices of the relevant samples
idx=[]
i=0
for (struct, center_i, center_j, species_i, species_j) in old_rho0ij.block(L=0).samples:
    
    if center_i==center_j: 
        idx.append(i)
    i+=1

In [20]:
old_rho0ij.block(L=0).values[idx]

array([[[ 0.01039805, -0.00505374,  0.0002431 ]],

       [[ 0.01039805, -0.00505374,  0.0002431 ]],

       [[ 0.01039805, -0.00505374,  0.0002431 ]],

       ...,

       [[ 0.01039805, -0.00505374,  0.0002431 ]],

       [[ 0.01039805, -0.00505374,  0.0002431 ]],

       [[ 0.01039805, -0.00505374,  0.0002431 ]]])

In [29]:
rho0ij.sparse()

Labels([( 0, 0, 0, 1, 1, 1), ( 0, 1, 0, 1, 1, 1), ( 0, 2, 0, 1, 1, 1),
        ( 1, 0, 0, 1, 1, 1), (-1, 0, 0, 1, 1, 1), ( 1, 1, 0, 1, 1, 1),
        (-1, 1, 0, 1, 1, 1), ( 1, 2, 0, 1, 1, 1), (-1, 2, 0, 1, 1, 1),
        ( 2, 0, 0, 1, 1, 8), ( 2, 1, 0, 1, 1, 8), ( 2, 2, 0, 1, 1, 8),
        ( 2, 0, 0, 1, 8, 1), ( 2, 1, 0, 1, 8, 1), ( 2, 2, 0, 1, 8, 1),
        ( 0, 0, 0, 1, 8, 8), ( 0, 1, 0, 1, 8, 8), ( 0, 2, 0, 1, 8, 8)],
       dtype=[('block_type', '<i4'), ('L', '<i4'), ('nu', '<i4'), ('sigma', '<i4'), ('species_i', '<i4'), ('species_j', '<i4')])

# acdc nu1 from acdc.ipynb

In [21]:
total_species = sorted(set(rhoi.sparse['center_species']))
# total_species = list(np.sort(np.asarray(total_species)))
lmax=rascal_hypers["max_angular"]
nmax=rascal_hypers["max_radial"]

In [22]:
blocks = []
for l in range(lmax+1):
    for sp_i in total_species:
        for sp_k in total_species:
            n_selected = nmax#len(np.where(opt_eva[l] > sel_thresh)[0])    
            de_block = rhoi.block(center_species = sp_i, neighbor_species=sp_k, spherical_harmonics_l = l)
            block = Block(
                values = de_block.values,
                samples = de_block.samples,
                components = Labels(["m"],np.asarray(range(-l,l+1), dtype=np.int32).reshape(-1,1)),
                features = Labels(["n"], np.asarray([[n] for n in range(nmax)], dtype=np.int32))
            )
            
            blocks.append( block )

In [23]:
block.components

Labels([(-2,), (-1,), ( 0,), ( 1,), ( 2,)], dtype=[('m', '<i4')])

In [24]:

acdc_nu1 = Descriptor(sparse = Labels(names=["L", "nu", "sigma","species_i", "neighbor_species"], 
                                      values=np.asarray([[ l, 1, 1, sp_i, sp_k] for l in range(rascal_hypers["max_angular"]+1) 
                                                        for sp_i in total_species
                                                        for sp_k in total_species], dtype=np.int32)
                                     ), 
                      blocks = blocks
                     )
#move neighbor species to features  
# acdc_nu1.sparse_to_features('neighbor_species')

In [25]:

acdc_nu1.block(L=2, nu=1, sigma=1, species_i=1, neighbor_species=8).components

Labels([(-2,), (-1,), ( 0,), ( 1,), ( 2,)], dtype=[('m', '<i4')])

# rho1ij

Basically we should build a function that takes a tensor product of rho0ij with any rhoi_nu to avoid having to keep implementing different orders of rho_ij. below we attempt to do this. 


**still work in progress**

In [26]:
nu_ij=rho0ij.sparse["nu"][0]

In [27]:
rho1 = acdc_nu1.sparse_to_features("neighbor_species")

In [35]:
def tensor_g_rho_nu(rho0ij, rhoinu, hypers, cg, feature_names=None):
    """ rho_ij^{nu+1} = <q|rho_i^{nu+1}; kh> <n| rho_ij^{0}; lm> cg 
    feature_names is a tuple of the form <n_rho0ij, l_rho0ij, n's in rhoi, k|
    Make sure you have transferred the species label to the feature (sparse_to_features)
    """
    nu_ij=rho0ij.sparse["nu"][0]
    
    # rhoinu = acdc_nu1; 
    feature_names=None
    nu_ij=rho0ij.sparse["nu"][0] #should be 0
    nu_rho= rhoinu.sparse["nu"][0]
    NU = nu_rho+nu_ij

    lmax= hypers["max_angular"]

    if cg is None:
        cg = ClebschGordanReal(lmax)

    sparse_labels=copy.deepcopy(rho0ij.sparse)
    sparse_labels["nu"] = NU

    new_sparse_labels = []
    for i in sparse_labels:
        new_sparse_labels.append(tuple(i))
        i[3]*=-1
        if i not in new_sparse_labels:
            new_sparse_labels.append(tuple(i))

    X_blocks = {new_sparse_labels[i]:[] for i in range(len(new_sparse_labels))}
    X_idx = {new_sparse_labels[i]:[] for i in range(len(new_sparse_labels))}
    X_samples= {new_sparse_labels[i]:[] for i in range(len(new_sparse_labels))}


    if feature_names is None:
        feature_names = (
            tuple(n + "_a" for n in rho0ij.block(0).features.names)
            + ("l_" + str(NU),)
            + tuple(n + "_b" for n in acdc_nu1.block(0).features.names)
            + ("k_" + str(NU),)
        )

    for index_a, block_a in rho0ij:
        block_type = index_a["block_type"]
        lam_a = index_a["L"]
        sigma_a = index_a["sigma"]
        sp_i, sp_j = index_a["species_i"], index_a["species_j"]
        for index_b, block_b in rhoinu:
            lam_b = index_b["L"]
            sigma_b = index_b["sigma"]
            rho_sp_i= index_b["species_i"]
            if not(sp_i== rho_sp_i):
                continue
            for L in range(np.abs(lam_a - lam_b),  min(lam_a + lam_b, lmax)+1):
                S = sigma_a * sigma_b * (-1) ** (lam_a + lam_b + L)
                block_idx=(block_type, L, NU, S, sp_i, sp_j)
                sel_feats=[]
                for n_a in range(len(block_a.features)):
                    f_a = tuple(block_a.features[n_a]) #values of n_a'th feature in block_a.features
                    for n_b in range(len(block_b.features)):
                        f_b = tuple(block_b.features[n_b]) #values of n_b'th feature in block_b.features
                        IDX = f_a  + (lam_a,)+f_b + (lam_b,)
                        sel_feats.append([n_a, n_b])
                        X_idx[block_idx].append(IDX)

                sel_feats = np.asarray(sel_feats, dtype=int)
                if len(sel_feats) == 0:
                    print(IDX, L, "len_feats 0")
                    continue

    #             #REMEMBER- values.shape = (nstruct*nat fitting the block criteria, 2*l+1, featsize)
                if block_type==0:
                    if not(sp_i== rho_sp_i):
                        continue
                    one_shot_blocks = cg.combine_einsum(
                     block_a.values[:, :, sel_feats[:, 0]],  #sel_feats[:,0]= n_a
                     block_b.values[:, :, sel_feats[:, 1]],  #sel_feats[:,1]= n_b*nspecies
                    L,
                    combination_string="iq,iq->iq",
                ) 

                    samples = block_a.samples

                elif block_type==1: #or block_type==-1: 
                    if not(sp_i== rho_sp_i):
                        continue
                    one_shot_blocks_ij = cg.combine_einsum(
                     block_a.values[:, :, sel_feats[:, 0]], 
                     block_b.values[0::2, :, sel_feats[:, 1]],
                    L,
                    combination_string="iq,iq->iq",
                )

                    one_shot_blocks_ji = cg.combine_einsum(
                     block_a.values[:, :, sel_feats[:, 0]], 
                     block_b.values[1::2, :, sel_feats[:, 1]],
                    L,
                    combination_string="iq,iq->iq",
                )
                    samples = block_a.samples
                    one_shot_blocks = (one_shot_blocks_ij+one_shot_blocks_ji)/np.sqrt(2)


                elif block_type==-1:
                    if not(sp_i== rho_sp_i):
                        continue
                    one_shot_blocks_ij = cg.combine_einsum(
                     block_a.values[:, :, sel_feats[:, 0]], 
                     block_b.values[0::2, :, sel_feats[:, 1]],
                    L,
                    combination_string="iq,iq->iq",
                )

                    one_shot_blocks_ji = cg.combine_einsum(
                     block_a.values[:, :, sel_feats[:, 0]], 
                     block_b.values[1::2, :, sel_feats[:, 1]],
                    L,
                    combination_string="iq,iq->iq",
                )
                    samples = block_a.samples
                    one_shot_blocks = (one_shot_blocks_ij-one_shot_blocks_ji)/np.sqrt(2)

                elif block_type==2:
                    #TODO: recheck this 
    #                     if sp_i<rho_sp_i:
    #                         continue
                    if not(sp_i== rho_sp_i):
                        continue
                    #print(sp_i, rho_sp_i, sp_j, block_a.values.shape, block_b.values.shape)
                    one_shot_blocks = cg.combine_einsum(
                     block_a.values[:, :, sel_feats[:, 0]], 
                     block_b.values[:, :, sel_feats[:, 1]],
                    L,
                    combination_string="iq,iq->iq",
                )
                    samples = block_a.samples

                else: 
                    print(block_type)
                    print("get the block type right you idiot")

                for Q in range(len(sel_feats)):
                    (n_a, n_b) = sel_feats[Q]
                    species_k, n = block_b.features[n_b]  #how to generalize this for nu
                    IDX = (n_a,)  + (lam_a,)+(species_k,)+(n,) + (lam_b,)
                    newblock = one_shot_blocks[:, :, Q]
#                     if(IDX not in X_idx[(block_type, L, NU, S, sp_i, sp_j)]):
#                         X_idx[(block_type, L, NU, S, sp_i, sp_j)].append(IDX)
   
                    X_blocks[block_idx].append(newblock)
                    X_samples[block_idx].append(samples)
    nonzero_idx = []

    nonzero_blocks = []
    for block_idx in X_blocks:
        block_type, L, NU, S, sp_i, sp_j = block_idx
        # create blocks
        if len(X_blocks[block_idx]) == 0:
            continue  # skips empty blocks

        nonzero_idx.append(block_idx)
        block_data = np.moveaxis(np.asarray(X_blocks[block_idx]), 0, -1) 
        block_samples = X_samples[block_idx][0]
    
        newblock = Block(
            values=block_data,
            samples=block_samples,
            components=Labels(
                ["mu"], np.asarray(range(-L, L + 1), dtype=np.int32).reshape(-1, 1)
            ),
            features=Labels(feature_names, np.asarray(X_idx[block_idx], dtype=np.int32)),
        )

        nonzero_blocks.append(newblock)
        print(block_idx, 'done')

    X = Descriptor(
        Labels(rho0ij.sparse.names, np.asarray(nonzero_idx, dtype=np.int32)), nonzero_blocks
    )


    return X

In [36]:
rho1ij=tensor_g_rho_nu(rho0ij, acdc_nu1, rascal_hypers, cg)

/Users/jigyasa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.


(0, 0, 1, 1, 1, 1) done
(0, 1, 1, 1, 1, 1) done
(0, 1, 1, -1, 1, 1) done
(0, 2, 1, 1, 1, 1) done
(0, 2, 1, -1, 1, 1) done
(1, 0, 1, 1, 1, 1) done
(-1, 0, 1, 1, 1, 1) done
(1, 1, 1, 1, 1, 1) done
(1, 1, 1, -1, 1, 1) done
(-1, 1, 1, 1, 1, 1) done
(-1, 1, 1, -1, 1, 1) done
(1, 2, 1, 1, 1, 1) done
(1, 2, 1, -1, 1, 1) done
(-1, 2, 1, 1, 1, 1) done
(-1, 2, 1, -1, 1, 1) done
(2, 0, 1, 1, 1, 8) done
(2, 1, 1, 1, 1, 8) done
(2, 1, 1, -1, 1, 8) done
(2, 2, 1, 1, 1, 8) done
(2, 2, 1, -1, 1, 8) done
(0, 0, 1, 1, 8, 8) done
(0, 1, 1, 1, 8, 8) done
(0, 1, 1, -1, 8, 8) done
(0, 2, 1, 1, 8, 8) done
(0, 2, 1, -1, 8, 8) done


In [34]:
rho1ij.block(0).

Labels([(  0, 1, 1), (  0, 2, 2), (  1, 1, 1), ..., (998, 2, 2),
        (999, 1, 1), (999, 2, 2)],
       dtype=[('structure', '<i4'), ('center_i', '<i4'), ('center_j', '<i4')])

# CRAP

In [58]:
def hamiltonian_rep(frames, hypers, nu):
    if nu>0:
        raise ValueError("nu>0 not implemented")
    nu0_desc = compute_rho0ij(hypers, frames)
    
    sparse = (block_type, center_sp, neighbor_sp, L, sigma)

In [313]:
gij_expansion=calculator.transform(ijframes).get_features(calculator).reshape(len(ijframes), len(ijf), max_atoms, hypers["max_radial"], -1)


In [389]:
feat_builder= DescriptorBuilder(["block_type", "L", "nu", "sigma",  "center_species", "neighbor_species"], ["structure", "center_i", "center_j"], ["mu"], ["n"])

pair_loc=[]
lmax = hypers["max_angular"]
for sp_i in actual_global_species:
    for sp_j in actual_global_species:
        center_species_mask = np.where(info[:, 2] == sp_i)[0]
        neighbor_species_mask = np.where(info[:, 2] == sp_j)[0]
        for i, (struct_i, atom_i) in enumerate(info[center_species_mask[:]][...,:-1]):
            for j, (struct_j, atom_j) in enumerate(info[neighbor_species_mask[:]][...,:-1]):
                if not (struct_i==struct_j):
                    continue
                if atom_i==atom_j:
                        block_type = 0  # diagonal
                elif sp_i==sp_j:
                    block_type = 1  # same-species
                else:
                    block_type = 2  # different species
                if [struct_i, atom_j, atom_i] not in pair_loc:
                    pair_loc.append([struct_i, atom_i, atom_j])

                for l in range(lmax+1):
                    block_idx=(block_type, l, 0, 1, sp_i, sp_j)
                    if block_idx not in feat_builder.blocks:
                        block = feat_builder.add_block(sparse=block_idx, features=np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T, 
                            components=np.asarray([list(range(-l, l+1))], dtype=np.int32 ).T )                                                    
#                         if block_type == 1:
#                             block_asym = feat_builder.add_block(sparse=(-1,)+block_idx[1:], features=np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T,
#                             components=np.asarray([list(range(-l, l+1))], dtype=np.int32 ).T )                                                    
                    else:                        
                        block = feat_builder.blocks[block_idx]
#                         if block_type == 1:
#                             block_asym = feat_builder.blocks[(-1,)+block_idx[1:]]

                    block_data =gij_expansion[struct_i][atom_i%3, atom_j%3, :, lm_slices[l]].T
#                     if block_type == 1:
#                         kj_offset = kj-kj_base
#                         ki_offset = ki-ki_base
#                         block_data_ji = np.asarray(ham[kj_base+ki_offset:kj_base+ki_offset+2*li+1, ki_base+kj_offset:ki_base+kj_offset+2*lj+1])                        
#                         block.add_samples(labels=[(A,i,j, ki_base, kj_base)], data=(block_data_ij+block_data_ji).reshape((1,-1,1))/np.sqrt(2) )
#                         block_asym.add_samples(labels=[(A,i,j,ki_base, kj_base)], data=(block_data_ij-block_data_ji).reshape((1,-1,1))/np.sqrt(2) )
#                     else:
                    block.add_samples(labels=np.asarray([[struct_i, atom_i%3, atom_j%3]], dtype=np.int32), data=block_data.reshape(1, -1, block_data.shape[1]))
#                     block.add_features(labels=[(n,)], data=block_data_ij.reshape((1,-1,1))

In [390]:
aa=feat_builder.build()

In [305]:
block.add_samples(labels=np.asarray([[struct_i, atom_i, atom_j]], dtype=np.int32), values=block_data)

In [249]:
blocks = []
for idx, (l, nu, sigma, sp_i, sp_j, ) in enumerate(sparse):
    center_species_mask = np.where(info[:, 2] == sp_i)[0]
    neighbor_species_mask = np.where(info[:, 2] == sp_j)[0]

    pair_loc=[]
    for i, (struct_i, atom_i) in enumerate(info[center_species_mask[:]][...,:-1]):
        for j, (struct_j, atom_j) in enumerate(info[neighbor_species_mask[:]][...,:-1]):
            if struct_i==struct_j:
                if [struct_i, atom_j, atom_i] not in pair_loc:
                    pair_loc.append([struct_i, atom_i, atom_j])
                    
    samples = Labels(
        names=["structure", "center_i", "center_j"],
        values=np.copy(pair_loc).astype(np.int32),
    )
    
    
    
    components = Labels(
                names=["mu"],
                values=np.array([[m] for m in range(-l, l + 1)], dtype=np.int32),
            )
    
    block_data = gij.block(spherical_harmonic_l=l).samples[]
    block = Block(
                values=np.copy(block_data),
                samples=samples,
                components=components,
#                 features=features,
            )
    blocks.append(block)
Descriptor(sparse, blocks)

TypeError: __init__() missing 2 required positional arguments: 'values' and 'features'

In [361]:
sp_j=1
sp_i=8
center_species_mask = np.where(info[:, 2] == sp_i)[0]
neighbor_species_mask = np.where(info[:, 2] == sp_j)[0]

In [364]:
pair_loc=[]
for i, (struct_i, atom_i) in enumerate(info[center_species_mask[:]][...,:-1]):
    for j, (struct_j, atom_j) in enumerate(info[neighbor_species_mask[:]][...,:-1]):
        if struct_i==struct_j:
            if [struct_i, int(atom_j), int(atom_i)] not in pair_loc:
                pair_loc.append([struct_i, int(atom_i), int(atom_j)])#+list(j[1:]))

In [ ]:
hypers=rascal_hypers

In [ ]:
#hypers = copy.deepcopy(self._hypers)
if hypers["compute_gradients"]:
    raise Exception("Pair expansion with gradient is not implemented")

# max_atoms = max([len(f) for f in frames])
# global_species = list(range(max_atoms))

# hypers["global_species"] = global_species
# hypers["expansion_by_species_method"] = "user defined"

# ijframes = []
# for f in frames:
#     ijf = f.copy()
#     ijf.numbers = global_species[:len(f)]
#     ijframes.append(ijf)

calculator = SphericalExpansion(**hypers)

# Step 2: move data around to follow the storage convention
sparse = Labels(
    names=["spherical_harmonics_l"],
    values=np.array(
        [
            [l]
            for l in range(hypers["max_angular"] + 1)                    
        ],
        dtype=np.int32,
    ),
)

features = Labels(
    names=["n"],
    values=np.array([[n] for n in range(hypers["max_radial"])], dtype=np.int32),
)

lm_slices = []
start = 0
for l in range(hypers["max_angular"] + 1):
    stop = start + 2 * l + 1
    lm_slices.append(slice(start, stop))
    start = stop

data = []
samples = []
for i, ijf in enumerate(ijframes):
    idata = calculator.transform(ijf).get_features(calculator).reshape(len(ijf), max_atoms, hypers["max_radial"], -1)
    nonzero = np.where( (idata**2).sum(axis=(2,3)) > 1e-20)
    data.append(idata[nonzero[0], nonzero[1]].reshape(len(nonzero[0]),hypers["max_radial"],-1) )
    samples.append( np.asarray( [nonzero[0]*0+i, nonzero[0], nonzero[1]] ).T )

data = np.concatenate(data)
samples = Labels(
        names=["structure", "center_i", "center_j"],
        values=np.concatenate(samples).astype(np.int32)
)
blocks = []
for (l,) in sparse:
    block_data = data[..., lm_slices[l]]
    block_data = block_data.swapaxes(1, 2)

    components = Labels(
        names=["spherical_harmonics_m"],
        values=np.array([[m] for m in range(-l, l + 1)], dtype=np.int32),
    )

    block = Block(
        values=np.copy(block_data),
        samples=samples,
        components=components,
        features=features,
    )

    blocks.append(block)
